In [13]:
# generic modules
from datetime import date
from typing import Final
import pandas as pd
import time
import sys

# local modules
from binance.client import Client
from binance.enums import *
from indicator import indicators

# local file
import secrets
from configparser import ConfigParser
import yfinance as yf
import numpy as np

In [21]:
config=ConfigParser()
config.read('keys.txt')

['keys.txt']

In [15]:
client = Client(str(config['KEYS']['binance_api_key']), str(config['KEYS']['binance_api_secret_key']))
print("Logged in successfully")

Logged in successfully


In [12]:
client.get_withdraw_history()


{'withdrawList': [{'amount': 0.09689584,
   'transactionFee': 0.006,
   'address': '0xcd1afdDC14d13d0BAA7E4Bd5293ccFba1515F2C3',
   'withdrawOrderId': '',
   'txId': '0xd2ee9ac6acfc00bb99ca88d378303a8de2192ff2cee3fbe7c5a9b89111e95d2b',
   'id': 'c11641e79ccc4c31af2a27ed1163c76e',
   'asset': 'ETH',
   'applyTime': 1621857012000,
   'status': 6,
   'network': 'ETH'}],
 'success': True}

In [4]:
position=""

In [18]:
def candle(symbol, interval="5m"):


    root_url = 'https://api.binance.com/api/v1/klines'
    url = root_url + '?symbol=' + symbol + '&interval=' + interval
    data = json.loads(requests.get(url).text)
    df = pd.DataFrame(data)
    df.columns = ['Datetime',
                'Open', 'High', 'Low', 'Close', 'volume',
                'close_time', 'qav', 'num_trades',
                'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.close_time]
    
    df.drop(['close_time','qav','num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'],axis=1,inplace=True)
           
    
    df['Open']=pd.to_numeric(df["Open"], downcast="float")
    df["High"]=pd.to_numeric(df["High"], downcast="float")
    df["Low"]=pd.to_numeric(df["Low"], downcast="float")
    df["Close"]=pd.to_numeric(df["Close"], downcast="float")
    df["volume"]=pd.to_numeric(df["volume"], downcast="float")
    
    
    
    
    df1=pd.DataFrame()
    Open=[]
    
    df1['Close']=((df['Open'] + df['High'] + df['Low'] + df['Close'])/4)
    for i in range(len(df)):
        if i==0:
            Open.append(0)

        else:
            Open.append((df['Open'][i-1]+df['Close'][i-1])/2)
    df1['Open']=np.array(Open)
    df1['volume']=df['volume']
    High=[]
    Low=[]
    High.append(0)
    Low.append(0)
    for i in range(1,len(df)):

        High.append(max(df['High'][i],df1['Close'][i],df1['Open'][i]))
        Low.append(min(df['Low'][i],df1['Close'][i],df1['Open'][i]))

    df1['High']=np.array(High)
    df1['Low']=np.array(Low)



    new_df = indicators.SuperTrend(df1, int(config['INPUTS']['atr']), int(config['INPUTS']['factor']))
    print(new_df[:-1])
    return new_df[:-1]


In [23]:
config['INPUTS']['atr']

'6'

In [6]:
def ltp_price(instrument):
    prices = client.get_all_tickers()
    for i in range(len(prices)):
        if prices[i]['symbol']==str(instrument):
            
            return float(prices[i]['price'])

    

In [7]:

def market_order():
    global position,price1,l
    price1=ltp_price(config['INPUTS']['symbol_binance'])
    order = client.create_order(
        symbol=str(config['INPUTS']['symbol_binance']),
        side=Client.SIDE_BUY,
        type=Client.ORDER_TYPE_MARKET,
       
        
        quantity=config['INPUTS']['quantity'])


    position='long'
    l=1




def market_order1():
    global position,price2,l
    price2=ltp_price(config['INPUTS']['symbol_binance'])
    order = client.create_order(
        symbol=str(config['INPUTS']['symbol_binance']),
        side=Client.SIDE_SELL,
        type=Client.ORDER_TYPE_MARKET,

        
        quantity=config['INPUTS']['quantity'])

    position='short'
    l=1



In [8]:
def main():
    times1=time.time()
    global quantity,df,position,price1,price2,l
    
    df=candle(config['INPUTS']['symbol_yahoo'])
    signal=""
    if l==0:
    
    
        if df.iloc[:,8][-1]!=df.iloc[:,8][-2] and position=="":
            if df.iloc[:,8][-1]=='down':
                signal='sell'

            if df.iloc[:,8][-1]=='up':
                signal='buy'
    ltp=ltp_price(config['INPUTS']['symbol_binance'])

    if l>0:
        if df.iloc[:,8][-1]!=df.iloc[:,8][-2] and position=="":
            if df.iloc[:,8][-1]=='down':
                signal='sell'

            if df.iloc[:,8][-1]=='up':
                signal='buy'

        if (position=="long" and df.iloc[:,8][-1]!=df.iloc[:,8][-2] and df.iloc[:,8][-1]=='down') or (position=="long" and ltp>price1+price1*(config['INPUTS']['take_profit_precentage'])/100) or(position=="long" and ltp<price1-price1*(config['INPUTS']['stop_loss_precentage'])/100):
            signal="squareoffsell"

        if (position=="short" and df.iloc[:,8][-1]!=df.iloc[:,8][-2] and df.iloc[:,8][-1]=='up') or (position=="short" and ltp<price2-price2*(config['INPUTS']['take_profit_precentage'])/100) or(position=="short" and ltp>price2+price2*(config['INPUTS']['stop_loss_precentage'])/100):
            signal="squareoffbuy"





    if signal=='buy':
        market_order()
        print("New long position initiated for ", config['INPUT']['symbol_binance'])


    elif signal=='sell':
        market_order1()
        print("New short position initiated for ", config['INPUT']['symbol_binance'])

    
    elif signal=="squareoffsell":
        market_order1()
        print("long position squared of for ", config['INPUT']['symbol_binance'])
        position=""

    elif signal=="squareoffbuy":
        market_order()
        print("short position squared of for ", config['INPUT']['symbol_binance'])
        position=""
 

    while True:
        if time.time()>=times1+300:
            break
        
        ltp=ltp_price(config['INPUTS']['symbol_binance'])
        if position=='long' and ((ltp>price1+price1*(config['INPUTS']['take_profit_precentage'])/100) or(ltp<price1-price1*(config['INPUTS']['stop_loss_precentage'])/100)):
            market_order1()
            position=""
        if position=='short' and ((ltp<price2-price2*(config['INPUTS']['take_profit_precentage'])/100) or(ltp>price2+price2*(config['INPUTS']['stop_loss_precentage'])/100)):
            market_order()
            position=""

        


In [9]:

l=0

while True:
        
        try:
                main()

        except:
                print("some exception ocurred... moving to next iteration")
                
   

